# In Depth A/B Testing - Lab

## Introduction

In this lab, you'll explore a survey from Kaggle regarding budding data scientists. With this, you'll form some initial hypotheses, and test them using the tools you've acquired to date. 

## Objectives

You will be able to:
* Conduct t-tests and an ANOVA on a real-world dataset and interpret the results

## Load the Dataset and Perform a Brief Exploration

The data is stored in a file called **multipleChoiceResponses_cleaned.csv**. Feel free to check out the original dataset referenced at the bottom of this lab, although this cleaned version will undoubtedly be easier to work with. Additionally, meta-data regarding the questions is stored in a file name **schema.csv**. Load in the data itself as a Pandas DataFrame, and take a moment to briefly get acquainted with it.

> Note: If you can't get the file to load properly, try changing the encoding format as in `encoding='latin1'`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
import flatiron_stats as fs
import warnings
warnings.filterwarnings("ignore")
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
#Your code here
df = pd.read_csv("multipleChoiceResponses_cleaned.csv")
df.head()

,GenderSelect,Country,Age,EmploymentStatus,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,...,JobFactorTitle,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity,exchangeRate,AdjustedCompensation
0,"Non-binary, genderqueer, or gender non-conforming",NaN,NaN,Employed full-time,NaN,NaN,Yes,NaN,DBA/Database Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Female,United States,30.0,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Somewhat important,NaN,NaN,NaN,NaN
2,Male,Canada,28.0,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,NaN,...,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,NaN,NaN
3,Male,United States,56.0,"Independent contractor, freelancer, or self-em...",NaN,NaN,Yes,NaN,Operations Research Practitioner,Poorly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,250000.0
4,Male,Taiwan,38.0,Employed full-time,NaN,NaN,Yes,NaN,Computer Scientist,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Wages and Education

You've been asked to determine whether education is impactful to salary. Develop a hypothesis test to compare the salaries of those with Master's degrees to those with Bachelor's degrees. Are the two statistically different according to your results?

> Note: The relevant features are stored in the 'FormalEducation' and 'AdjustedCompensation' features.

You may import the functions stored in the `flatiron_stats.py` file to help perform your hypothesis tests. It contains the stats functions that you previously coded: `welch_t(a,b)`, `welch_df(a, b)`, and `p_value(a, b, two_sided=False)`. 

Note that `scipy.stats.ttest_ind(a, b, equal_var=False)` performs a two-sided Welch's t-test and that p-values derived from two-sided tests are two times the p-values derived from one-sided tests. See the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) for more information.    

In [3]:
#Your code here
df_MD = df[df["FormalEducation"]=="Master's degree"]["AdjustedCompensation"].dropna()
df_BD = df[df["FormalEducation"]=="Bachelor's degree"]["AdjustedCompensation"].dropna()
print("Master's degree - Minimum:", df_MD.min(), "\n\t\t  Mean:", df_MD.mean(), "\n\t\t  Max:", df_MD.max())
print("Bachelor's degree - Minimum:", df_BD.min(), "\n\t\t    Mean:", df_BD.mean(), "\n\t\t    Max:", df_BD.max())

Master's degree - Minimum: 0.0 
		  Mean: 69139.89987120005 
		  Max: 4498900.0
Bachelor's degree - Minimum: 0.0 
		    Mean: 64887.09799461879 
		    Max: 9999999.0


In [4]:
print(df_MD.describe())
print(df_BD.describe())

count    1.990000e+03
mean     6.913990e+04
std      1.355272e+05
min      0.000000e+00
25%      2.186800e+04
50%      5.381217e+04
75%      9.200000e+04
max      4.498900e+06
Name: AdjustedCompensation, dtype: float64
count    1.107000e+03
mean     6.488710e+04
std      3.069359e+05
min      0.000000e+00
25%      1.126620e+04
50%      3.839940e+04
75%      8.023100e+04
max      9.999999e+06
Name: AdjustedCompensation, dtype: float64


In [5]:
fs.p_value_welch_ttest(df_MD, df_BD)

0.33077639451272267

In [6]:
fs.p_value_welch_ttest(df_MD[(df_MD > 0) & (df_MD < 1000000)], df_BD[(df_BD > 0) & (df_BD < 1000000)])

7.931617733714091e-07

Removing null values only, they are not significantly different, but also removing outliers by setting a range between 0 and 1,000,000 shows a significant difference.

## Wages and Education II

Now perform a similar statistical test comparing the AdjustedCompensation of those with Bachelor's degrees and those with Doctorates. If you haven't already, be sure to explore the distribution of the AdjustedCompensation feature for any anomalies. 

In [7]:
#Your code here
df_DD = df[df["FormalEducation"]=="Doctoral degree"]["AdjustedCompensation"].dropna()
df_BD = df[df["FormalEducation"]=="Bachelor's degree"]["AdjustedCompensation"].dropna()

print("Median Values: \nDoctoral:", df_DD.median(), "\nBachelor's:", df_BD.median())
print("Sample sizes: \nDoctoral:", df_DD.count(), "\nBachelor's:", df_BD.count())
print("Welch's t-test p-value:", fs.p_value_welch_ttest(df_DD, df_BD))

df_DD = df_DD[(df_DD > 0) & (df_DD < 1000000)]
df_BD = df_BD[(df_BD > 0) & (df_BD < 1000000)]

print("\nRepeated Test with Ouliers Removed:")
print("Sample sizes: \nDoctoral:", df_DD.count(), "\nBachelor's:", df_BD.count())
print("Welch's t-test p-value with outliers removed:", fs.p_value_welch_ttest(df_DD, df_BD))

Median Values: 
Doctoral: 74131.91999999997 
Bachelor's: 38399.4
Sample sizes: 
Doctoral: 967 
Bachelor's: 1107
Welch's t-test p-value: 0.1568238199472023

Repeated Test with Ouliers Removed:
Sample sizes: 
Doctoral: 959 
Bachelor's: 1096
Welch's t-test p-value with outliers removed: 0.0


## Wages and Education III

Remember the multiple comparisons problem; rather than continuing on like this, perform an ANOVA test between the various 'FormalEducation' categories and their relation to 'AdjustedCompensation'.

In [8]:
#Your code here
formula = "AdjustedCompensation ~ C(FormalEducation)"
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

lm = ols(formula, df[(df.AdjustedCompensation > 0) & (df.AdjustedCompensation < 1000000)]).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

                          sum_sq      df         F    PR(>F)
C(FormalEducation)  6.540294e+17     6.0  0.590714  0.738044
Residual            7.999414e+20  4335.0       NaN       NaN
                          sum_sq      df          F        PR(>F)
C(FormalEducation)  5.548565e+11     6.0  26.127919  1.082559e-30
Residual            1.515907e+13  4283.0        NaN           NaN


## Additional Resources

Here's the original source where the data was taken from:  
    [Kaggle Machine Learning & Data Science Survey 2017](https://www.kaggle.com/kaggle/kaggle-survey-2017)

## Summary

In this lab, you practiced conducting actual hypothesis tests on actual data. From this, you saw how dependent results can be on the initial problem formulation, including preprocessing!